In [5]:
import numpy as np

def net(node, inp):
    mul = None
    if node == 0 :
        mul = (w[0] * inp[0]) + (w[2] * inp[1])
    if node == 1 :
        mul = (w[1] * inp[0]) + (w[3] * inp[1])
    if node == 2 :
        mul = (w[4] * h[0]) + (w[5] * h[1])
    if node == 3 :
        mul = (w[6] * h[0]) + (w[7] * h[1])
    return mul + b[node // 2]

def act_func(val):
    return 1 / (1 + np.exp(-val))

def calc_error(out, d_out):
    e1 = 0.5 * ((out[0] - d_out[0]) ** 2)
    e2 = 0.5 * ((out[1] - d_out[1]) ** 2)
    return e1 + e2

def cal_del_w(i, w, inp, out, d_out):
    # Precompute common error terms for the outputs:
    delta_out0 = (out[0] - d_out[0]) * (out[0] * (1 - out[0]))
    delta_out1 = (out[1] - d_out[1]) * (out[1] * (1 - out[1]))

    if i == 0:
        # Weight from input[0] to hidden[0]
        delta_hidden0 = (delta_out0 * w[4] + delta_out1 * w[6]) * (h[0] * (1 - h[0]))
        return delta_hidden0 * inp[0]
    elif i == 1:
        # Weight from input[0] to hidden[1]
        delta_hidden1 = (delta_out0 * w[5] + delta_out1 * w[7]) * (h[1] * (1 - h[1]))
        return delta_hidden1 * inp[0]
    elif i == 2:
        # Weight from input[1] to hidden[0]
        delta_hidden0 = (delta_out0 * w[4] + delta_out1 * w[6]) * (h[0] * (1 - h[0]))
        return delta_hidden0 * inp[1]
    elif i == 3:
        # Weight from input[1] to hidden[1]
        delta_hidden1 = (delta_out0 * w[5] + delta_out1 * w[7]) * (h[1] * (1 - h[1]))
        return delta_hidden1 * inp[1]
    elif i == 4:
        # Weight from hidden[0] to out[0]
        return delta_out0 * h[0]
    elif i == 5:
        # Weight from hidden[1] to out[0]
        return delta_out0 * h[1]
    elif i == 6:
        # Weight from hidden[0] to out[1]
        return delta_out1 * h[0]
    elif i == 7:
        # Weight from hidden[1] to out[1]
        return delta_out1 * h[1]

In [6]:
import random
w = [random.randint(0,1) for i in range(8)]
inp = [[0,0], [0,1], [1,0], [1,1]]
d_out = [[0,1], [1,0], [1,0], [0,1]]
b = [0.25, 0.5]
h = [act_func(net(0, inp[0])), act_func(net(1, inp[0]))]
out = [[act_func(net(2, inp[0])), act_func(net(3, inp[0]))],
       [act_func(net(2, inp[1])), act_func(net(3, inp[1]))],
       [act_func(net(2, inp[2])), act_func(net(3, inp[2]))],
       [act_func(net(2, inp[3])), act_func(net(3, inp[3]))]]

In [7]:
def step(k, i, w, out, h, lr):
    del_w = np.zeros(8)
    for j in range(7, -1, -1):
        del_w[j] = lr * cal_del_w(j, w, inp[i], out[i], d_out[i])
    out[i][0] = act_func(net(2, inp[i]))
    out[i][1] = act_func(net(3, inp[i]))
    h[0] = act_func(net(0, inp[i]))
    h[1] = act_func(net(1, inp[i]))
    if k % 10000 == 1:
        print(f"Epoch({k}), Step({i+1}) Out1 = {out[i][0]} Out2 = {out[i][1]} Error = {calc_error(out[i], d_out[i])}")
    return del_w

In [8]:
def epoch(epochs, lr):
    global w
    e = [0,0,0,0]
    for j in range(epochs):
        sum_w = np.zeros(8)
        for i in range(4):
            sum_w += step(j, i, w, out, h, lr)
            e[i] = calc_error(out[i], d_out[i])
        w -= sum_w
        if all(error < 0.2 for error in e):
            print("Training stopped early as all errors are less than 0.1")

epoch(40000, lr=0.2)

Epoch(1), Step(1) Out1 = 0.8321624971345067 Out2 = 0.8321624971345067 Error = 0.3603319245026312
Epoch(1), Step(2) Out1 = 0.8321624971345067 Out2 = 0.8321624971345067 Error = 0.3603319245026312
Epoch(1), Step(3) Out1 = 0.8315498315960583 Out2 = 0.8315498315960583 Error = 0.3599252908313746
Epoch(1), Step(4) Out1 = 0.8315498315960583 Out2 = 0.8315498315960583 Error = 0.3599252908313746
Epoch(10001), Step(1) Out1 = 0.622463303288331 Out2 = 0.6224535896908717 Error = 0.2650009279389647
Epoch(10001), Step(2) Out1 = 0.9670466360729142 Out2 = 0.025045572949829453 Error = 0.0008566024592480946
Epoch(10001), Step(3) Out1 = 0.6243300780997498 Out2 = 0.6197488001026795 Error = 0.2626082327246255
Epoch(10001), Step(4) Out1 = 0.6257169217050363 Out2 = 0.6177310774816451 Error = 0.2688255976156853
Epoch(20001), Step(1) Out1 = 0.6224600846506725 Out2 = 0.6224582903114594 Error = 0.26499714976893435
Epoch(20001), Step(2) Out1 = 0.9791071747000674 Out2 = 0.01591884047790368 Error = 0.00034495981558722

In [9]:
def calc_out(inp, out):
    print("x\ty\tXOR")
    for i in range(4):
        print(f"{inp[i][0]}\t{inp[i][1]}\t{out[i][0] }\t{out[i][1]}")

calc_out(inp, out)

x	y	XOR
0	0	0.6224594986256513	0.622459107143182
0	1	0.9860887159935301	0.010630889173392605
1	0	0.6229143602112193	0.6218500408284579
1	1	0.6232010440958897	0.621465820160717
